In [1]:
import subprocess
import sys
import requests
import re
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

# Make abstract

In [ ]:
class NHLGameIdPipeline:
    # Define allowed game types with descriptions
    GAME_TYPE_OPTIONS = {
        1: "PreSeason",
        2: "RegSeason",
        3: "Playoffs"
    }

    def __init__(self, team_code, season, game_type):
        self.team_code = team_code
        self.season = season
        self.data = None
        self.flattened_data = []
        # Validate and set game_type
        self.game_type = self.validate_game_type(game_type)

    def validate_game_type(self, game_type):
        """Validate the game type and display options if invalid."""
        if game_type not in self.GAME_TYPE_OPTIONS:
            print("Invalid game type!")
            # for code, name in self.GAME_TYPE_OPTIONS.items():
            #     print(f"{code}: {name}")
            raise ValueError("Please enter a valid game type.")
        return game_type

    def fetch_data(self):
        """Fetch data from the NHL API for the specified team and season."""
        url = f"https://api-web.nhle.com/v1/club-schedule-season/{self.team_code}/{self.season}" #This is a public API from the NHL.
        response = requests.get(url)
        if response.status_code == 200:
            self.data = response.json().get('games', [])
        else:
            print(f"Failed to fetch data: {response.status_code}")
        return self

    def filter_games_by_type(self):
        """Filter games to include only those of the specified game type."""
        self.data = [game for game in self.data if game.get('gameType') == self.game_type]
        return self

    def flatten_data(self):
        """Flatten each game dictionary and store in flattened_data."""
        for game in self.data:
            if isinstance(game, dict):
                flat_game = {
                    'id': game.get('id'),
                    'season': game.get('season'),
                    'gameDate': game.get('gameDate'),
                    'gameType': game.get('gameType'),
                    'homeTeam_id': game['homeTeam'].get('id'),
                    'homeTeam_placeName': game['homeTeam']['placeName'].get('default'),
                    'homeTeam_abbrev': game['homeTeam'].get('abbrev'),
                    'homeTeam_score': game['homeTeam'].get('score'),
                    'awayTeam_id': game['awayTeam'].get('id'),
                    'awayTeam_placeName': game['awayTeam']['placeName'].get('default'),
                    'awayTeam_abbrev': game['awayTeam'].get('abbrev'),
                    'awayTeam_score': game['awayTeam'].get('score')
                }
                self.flattened_data.append(flat_game)
        return self

    def to_dataframe(self):
        """Convert the flattened data into a DataFrame."""
        return pd.DataFrame(self.flattened_data)

    def to_postgres(self, table_name, db_url):
        """Export the flattened data DataFrame to a PostgreSQL database."""
        # Ensure data is in a DataFrame
        df = pd.DataFrame(self.flattened_data)

        # Create SQLAlchemy engine
        engine = create_engine(db_url)

        # Write DataFrame to SQL
        with engine.connect() as connection:
            df.to_sql(table_name, con=connection, if_exists='replace', index=False)
        print(f"Data successfully written to {table_name} table in PostgreSQL.")

    @classmethod
    def display_game_type_options(cls):
        """Display the valid game type options."""
        print("Please choose a valid game type from the following options:")
        for code, name in cls.GAME_TYPE_OPTIONS.items():
            print(f"{code}: {name}")

In [ ]:
try:
    TEAMCODE = "VAN"
    SEASON = "20232024"
    GAMETYPE = 2

    pipeline = NHLGameIdPipeline(TEAMCODE, SEASON, GAMETYPE)
    df = (pipeline
          .fetch_data()
          .filter_games_by_type()
          .flatten_data()
          .to_dataframe())
    display(df.head())
except ValueError:
    NHLGameIdPipeline.display_game_type_options()

,id,season,gameDate,gameType,homeTeam_id,homeTeam_placeName,homeTeam_abbrev,homeTeam_score,awayTeam_id,awayTeam_placeName,awayTeam_abbrev,awayTeam_score
0,2023020009,20232024,2023-10-11,2,23,Vancouver,VAN,8,22,Edmonton,EDM,1
1,2023020029,20232024,2023-10-14,2,22,Edmonton,EDM,3,23,Vancouver,VAN,4
2,2023020042,20232024,2023-10-17,2,4,Philadelphia,PHI,2,23,Vancouver,VAN,0
3,2023020055,20232024,2023-10-19,2,14,Tampa Bay,TBL,4,23,Vancouver,VAN,3
4,2023020068,20232024,2023-10-21,2,13,Florida,FLA,3,23,Vancouver,VAN,5


# Exporting to Neon Postgress DataBse

In [ ]:
def to_postgres(df, table_name, db_url):
        """Export the flattened data DataFrame to a PostgreSQL database."""
        # Create SQLAlchemy engine
        engine = create_engine(db_url)

        # Write DataFrame to SQL
        with engine.connect() as connection:
            df.to_sql(table_name, con=connection, if_exists='replace', index=False)
        print(f"Data successfully written to {table_name} table in PostgreSQL.")

In [ ]:
# PostgreSQL connection URL for Neon
DATABASE_URL= #Give a String for your DB URL
pipeline.to_postgres(table_name="nhl_games", db_url=DATABASE_URL)

Data successfully written to nhl_games table in PostgreSQL.


In [ ]:
df

,id,season,gameDate,gameType,homeTeam_id,homeTeam_placeName,homeTeam_abbrev,homeTeam_score,awayTeam_id,awayTeam_placeName,awayTeam_abbrev,awayTeam_score
0,2023020009,20232024,2023-10-11,2,23,Vancouver,VAN,8,22,Edmonton,EDM,1
1,2023020029,20232024,2023-10-14,2,22,Edmonton,EDM,3,23,Vancouver,VAN,4
2,2023020042,20232024,2023-10-17,2,4,Philadelphia,PHI,2,23,Vancouver,VAN,0
3,2023020055,20232024,2023-10-19,2,14,Tampa Bay,TBL,4,23,Vancouver,VAN,3
4,2023020068,20232024,2023-10-21,2,13,Florida,FLA,3,23,Vancouver,VAN,5
...,...,...,...,...,...,...,...,...,...,...,...,...
77,2023021238,20232024,2024-04-08,2,23,Vancouver,VAN,4,54,Vegas,VGK,3
78,2023021254,20232024,2024-04-10,2,23,Vancouver,VAN,3,53,Arizona,ARI,4
79,2023021280,20232024,2024-04-13,2,22,Edmonton,EDM,1,23,Vancouver,VAN,3
80,2023021302,20232024,2024-04-16,2,23,Vancouver,VAN,4,20,Calgary,CGY,1


# Get Player Stat based on Gamecode

In [ ]:
class NHLGameStatsPipeline:
    def __init__(self, game_code):
        self.game_code = game_code
        self.data = None
        self.flattened_data = []

    def fetch_data(self):
        """Fetch data from the NHL API for the specified game code."""
        url = f"https://api-web.nhle.com/v1/gamecenter/{self.game_code}/boxscore"
        response = requests.get(url)
        if response.status_code == 200:
            self.data = response.json()
        else:
            print(f"Failed to fetch data: {response.status_code}")
        return self

    def clean_data(self):
        """Clean the fetched data."""
        toKeep = ['id', 'season',  'gameDate', 'playerByGameStats']
        for key in list(self.data.keys()):
            if key not in toKeep:
              self.data.pop(key)
        return self

    def flatten_data(self):
        """Flatten each game dictionary and store in flattened_data."""

        dateInfo = {
            'id': self.data['id'],
            'season': self.data['season'],
            'gameDate': self.data['gameDate']
        }

        for HA, HAvalue in self.data['playerByGameStats'].items():
            for PosKey, PosKeyValue in HAvalue.items():
                for x in range(len(PosKeyValue)):
                    PosKeyValue[x]['name'] = PosKeyValue[x]['name']['default']
                    if re.match(r'^[LR]$' , PosKeyValue[x]['position']):
                        PosKeyValue[x]['position'] += 'W'

            HAvalue['All'] = HAvalue['forwards'] + HAvalue['defense'] + HAvalue['goalies']
            HAvalue.pop('forwards')
            HAvalue.pop('defense')
            HAvalue.pop('goalies')
            for x in range(len(HAvalue['All'])):
                HAvalue['All'][x].update({'HA': HA})
                HAvalue['All'][x].update(dateInfo)
        return self

    def to_dataframe(self):
        """Convert the flattened data into a DataFrame."""
        gameData = []
        for HA, HAvalue in self.data['playerByGameStats'].items():
            for x in range(len(HAvalue['All'])):
                gameData.append(HAvalue['All'][x])

        df = pd.DataFrame(gameData)
        cols_to_move = ['id', 'season', 'gameDate']
        df = df[ cols_to_move + [ col for col in df.columns if col not in cols_to_move ] ]
        df.fillna(0, inplace=True)
        return df

In [ ]:
data = NHLGameStatsPipeline("2023020009").fetch_data().clean_data().flatten_data().to_dataframe()

In [ ]:
DATABASE_URL= #Give a String for your DB URL.
to_postgres(data, table_name="20232024Game1", db_url=DATABASE_URL)

Data successfully written to 20232024Game1 table in PostgreSQL.
